In [39]:
import os
import json
import pandas as pd
import re

from openai import OpenAI

## 데이터 전처리

In [2]:
# 엑셀 파일 경로
file_path = '../data/arts_all.xlsx'
df = pd.read_excel(file_path)
df = df[df['작품이미지'] == 1].reset_index(drop=True)
df["이미지"] = df['번호'].apply(lambda x: str(x)+".jpg")
# df = df[0:50]
df.to_csv("../data/arts_all.csv", index=False)

In [46]:
df = pd.read_csv("../data/arts_all.csv")

### Solar API
- 작가의 설명과 작품의 설명을 분리합니다.

In [72]:
prompt = """
## Role: 미술 작품 설명 추출기

## Instruction
- 미술 작품에 대한 설명이 주어지면 작가에 대한 설명과 작품에 대한 설명을 분리하고 요약한다.
- 아래 Json 포맷에 맞게 답변을 생성한다.
- 한국어로 생성한다.

## Output format
{"art_description": "$art_description", "author_description":"$author_description"}
"""

for i in range(len(df)):
    row_data = df.loc[i]
    content = df['작품 설명'][i]

    client = OpenAI(
        api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
    )
    chat_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": content},
        ],
    )

    try:
        output_dict = json.loads(chat_result.choices[0].message.content)
        df.loc[i, 'art_description'] = output_dict['art_description']
        df.loc[i, 'author_description'] = output_dict['author_description']
    except:
        df.loc[i, 'art_description'] = "Error"
        df.loc[i, 'author_description'] = "Error"
    

In [133]:
df.to_csv("../data/arts_all02.csv", index=False)

In [138]:
df = pd.read_csv("../data/arts_all02.csv")

In [140]:
df

,번호,작가,작품명,제작연도,재료,규격,부문,관리번호,작품 설명,작품이미지,이미지,art_description,author_description
0,1,이중섭,포도 따는 남자,194,"종이에 펜, 수채 물감",14 x 9,드로잉,10090.0,"대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에...",1,1.jpg,이 작품은 자신의 몸보다 큰 포도 아래 쭈그린 자세로 앉아있는 인물을 그린 작품입니...,"대향(大鄕) 이중섭(李仲燮, 1916-1956)은 평안북도 정주의 오산고등보통학교에..."
1,2,김기창,진돗개,1975,"종이에 먹, 색",66.3 x 95.6,회화1,9172.0,"운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장...",1,2.jpg,조화로운 색채와 적절한 먹의 농담 표현으로 개의 모습을 생동감 있게 전달한 작품으로...,"운보(雲甫) 김기창(金基昶, 1914-2001)은 청각장애를 갖고 있으며, 이당(以..."
2,3,김기창,청산정경,1979,"비단에 먹, 색",68.2 x 136.6,회화1,9177.0,"운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장...",1,3.jpg,"청산정경은 김기창의 청록산수화로, 화면에는 군청색과 녹청색 계열로 채색한 큰 산을 ...",김기창은 어렸을 때 장티푸스에 걸려 청각장애를 갖게 되었습니다. 17세에 이당 김은...
3,4,이응노,구상,1974,"캔버스에 천, 콜라주",204 x 126,회화1,9909.0,"고암(顧庵) 이응노(李應魯, 1904-1989)는 1922년에 동양화가인 김규진에게...",1,4.jpg,이 작품은 거친 천 위에 다른 색 천으로 만든 문자 형상을 붉은색 실로 꿰맨 것처럼...,"이응노는 한국의 화가이며, 전통 회화와 서양화를 모두 공부했습니다. 그는 1945년..."
4,5,이응노,사자,1976,"종이에 먹, 색",42.7 x 59.5,회화1,9927.0,"고암(顧菴) 이응노(李應魯, 1904-1989)는 17세 무렵 서화가 송태회(宋泰會...",1,5.jpg,이 작품은 이응노 특유의 강한 서체적 필선과 산뜻한 담채를 통해 강인한 사자의 모습...,"고암(顧菴) 이응노(李應魯, 1904-1989)는 17세 무렵 서화가 송태회(宋泰會..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,119,한스 하케 Hans HAACKE,아이스 테이블,1967,"스테인리스 스틸, 냉동 장치",91.4×92×49,조각ㆍ설치,8202.0,한스 하케(1936- )는 개념미술가로서 제도비판미술의 한 전형을 제시한 작가로 잘...,1,119.jpg,한스 하케의 개념미술은 제도비판미술의 한 전형을 제시했습니다. 그의 작품은 자연 에...,한스 하케(1936-)는 독일의 개념미술가입니다. 그는 1950년대 참여했던 제로 ...
64,120,왈리드 라드 Walid RAAD,아홉번째 판에 부치는 서문: 마완 카삽-바치(1934-2016),2017,"29개의 프레임과 드로잉, 나무 벽, 벽지",가변크기,드로잉,8142.0,왈리드 라드(1967- )는 레바논 전쟁에 대한 역사적인 자료와 상상에 의해 가공된...,1,120.jpg,"왈리드 라드의 작품은 사진, 영화, 퍼포먼스 등 다양한 매체를 활용하며 1975년부...",왈리드 라드(1967- )는 레바논 전쟁에 대한 역사적인 자료와 상상에 의해 가공된...
65,129,파블로 피카소 Pablo PICASSO,큰 새와 검은 얼굴,1951,"백토, 화장토 장식, 나이프 각인",50×47×38,공예,10460.0,"파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Mál...",1,129.jpg,큰 새와 검은 얼굴은 검은 줄무늬가 있는 새와 그것의 가슴 부분에 검은 얼굴이 묘사...,"파블로 피카소는 스페인의 화가로, 청색 시대와 장밋빛 시대를 거쳐 큐비즘을 창시하였..."
66,130,파블로 피카소 Pablo PICASSO,자클린의 옆모습,1956,"백토, 흑색 파티나 장식, 부분 유약 시유",19×19×2.5,공예,10360.0,"파블로 피카소(Pablo Picasso, 1881–1973)는 스페인 말라가(Mál...",1,130.jpg,"자클린의 옆모습은 피카소가 1953년에 만든 도자기 작품으로, 그의 두 번째 아내인...",파블로 피카소는 1881년 스페인 말라가에서 태어나 1973년 프랑스 무진에서 사망...
